In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
ENTREPOT_PATH = '~/Bureau/utils/data/'
METEO_PATH = '~/Bureau/utils/data/meteo/'
SPATIAL_PATH = './data/spatial/'

In [5]:
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(" - ", df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'noeuds_synthetise', 'connection_synthetise', 'synthetise', 'culture', 'noeuds_synthetise_restructure'
]

tables_performance = [
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',',index_col='id', verbose=False)

# import des données de performance
#import_dfs(['intervention_realise_agrege', 'combinaison_outil_materiel'], ENTREPOT_PATH, sep = ',', verbose=False)


100%|██████████| 5/5 [00:01<00:00,  2.76it/s]


In [6]:
# INVENTAIRE DES DIFFÉRENTS SCÉNARIOS : 
# A : fr.inra.agrosyst.api.entities.practiced.PracticedSystem_000f911f-5e67-4280-ae77-35098c17aa5d
# B : fr.inra.agrosyst.api.entities.practiced.PracticedSystem_00b5a0b4-39a6-4822-802b-e81fd44386a2, fr.inra.agrosyst.api.entities.practiced.PracticedSystem_01b520b2-0104-44b0-ac88-e297027a6037
# C : fr.inra.agrosyst.api.entities.practiced.PracticedSystem_05e5d9d6-ad2b-43b2-b37a-1f32850e37a1
# D : fr.inra.agrosyst.api.entities.practiced.PracticedSystem_0316e326-6369-4ec7-ab50-b983c3aa0b3d
# E : fr.inra.agrosyst.api.entities.practiced.PracticedSystem_1e606337-4238-437b-9856-a302b2431efd
# F : fr.inra.agrosyst.api.entities.practiced.PracticedSystem_0b41d014-7b0d-42ea-b8cd-d43d58b270ad

In [31]:
studied_synthetise_id = [
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_000f911f-5e67-4280-ae77-35098c17aa5d',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_00b5a0b4-39a6-4822-802b-e81fd44386a2',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_05e5d9d6-ad2b-43b2-b37a-1f32850e37a1',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_0316e326-6369-4ec7-ab50-b983c3aa0b3d',
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_1e606337-4238-437b-9856-a302b2431efd', 
]

In [32]:
df['synthetise_test'] = df['synthetise'].loc[
    df['synthetise'].index.isin(studied_synthetise_id)
]

In [33]:
df['noeuds_synthetise_test'] = df['noeuds_synthetise'].loc[
    df['noeuds_synthetise']['synthetise_id'].isin(df['synthetise_test'].index)
]

In [34]:
df['connection_synthetise_test'] = df['connection_synthetise'].loc[
    (df['connection_synthetise']['source_noeuds_synthetise_id'].isin(df['noeuds_synthetise_test'].index)) |
    (df['connection_synthetise']['cible_noeuds_synthetise_id'].isin(df['noeuds_synthetise_test'].index))
]

In [35]:
left = df['noeuds_synthetise']
right = df['noeuds_synthetise_restructure']
df['noeuds_synthetise_extanded'] = pd.merge(left, right, left_index=True, right_index=True, how='left')

In [36]:
left = df['noeuds_synthetise_extanded']
right = df['culture']
df['noeuds_synthetise_extanded'] = pd.merge(left, right, left_on='culture_id', right_index=True, how='left')


In [37]:
left = df['connection_synthetise_test']
right = df['noeuds_synthetise_extanded'].rename(columns={'nom' : 'nom_culture_cible'})
merge = pd.merge(left, right, left_on='cible_noeuds_synthetise_id', right_index=True, how='left')

left = merge
right = df['noeuds_synthetise_extanded'][['nom']].rename(columns={'nom' : 'nom_culture_source'})
merge = pd.merge(left, right, left_on='source_noeuds_synthetise_id', right_index=True, how='left')

In [38]:
merge[['rang', 'nom_culture_source', 'nom_culture_cible', 'synthetise_id']]

,rang,nom_culture_source,nom_culture_cible,synthetise_id
id,,,,
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_331c36ff-3258-432a-8a8b-b05f2207c908,2,GRAMINEES FOURRAGERES,MAIS ENSILAGE,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_bbde3d98-8744-4b95-a8ea-db9a31f936f8,0,MAIS ENSILAGE,BLE TENDRE D'HIVER,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_a45db8c4-6af2-42ef-9d9e-32d7281692ca,0,MAIS ENSILAGE,BLE TENDRE D'HIVER,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_f5832097-5d13-4871-9d15-fcb643f539a5,1,BLE TENDRE D'HIVER,GRAMINEES FOURRAGERES,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_ccf2d625-0302-4325-90df-4aea9c251e8c,1,BLE TENDRE D'HIVER,MAIS ENSILAGE,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_c6fa5afb-2258-401d-ac69-342fc0554589,1,artichaut,salade,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_acab5249-51ee-4a90-baf2-bb3b40c055c9,2,salade,salade,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_6a366c47-f873-4992-8dc5-20a872e5707f,0,salade,artichaut,fr.inra.agrosyst.api.entities.practiced.Practi...
fr.inra.agrosyst.api.entities.practiced.PracticedCropCycleConnection_c64570ec-7563-47b6-afaf-13c4a71727fe,1,prairie temporaire,Maïs,fr.inra.agrosyst.api.entities.practiced.Practi...


In [40]:
path=''
df['connection_synthetise_test'].to_csv(path+'connection_synthetise'+'.csv')
df['noeuds_synthetise_test'].to_csv(path+'noeuds_synthetise'+'.csv')
